## Geoampify Feature Extraction

In [ ]:
# # Code formatter
# # !pip3 install nb_black
# %load_ext nb_black

In [1]:
# Import required libraries

# eda tools
import pandas as pd
pd.options.display.max_columns = 100

# api tools
import requests
import json

# Import the API key
from config import geoapify_key

# hide jupyter lab warnings
import warnings
warnings.filterwarnings('ignore')


# make sound when this code executes: Audio(sound_file, autoplay=True)
from IPython.display import Audio
sound_file = './sound/chord.wav'


In [2]:
# display package informatin
# !conda install -c conda-forge session-info
import session_info
session_info.show()

### Read Dataset

In [3]:
geoapify_key

'89e77e8eb6fa4e2b930e7419dcbe0de0'

In [4]:
# Read data
restaurant_df = pd.read_csv('./data/manipulated/combined_data.csv')
grid_layout = pd.read_csv('./data/manipulated/chicago_point_grid.csv')

# View key file
grid_layout.head()

,lat,lon,geometry
0,-87.640611,41.661182,POINT (-87.64061065791103 41.66118220092816)
1,-87.607332,41.661182,POINT (-87.60733249962075 41.66118220092816)
2,-87.590693,41.661182,POINT (-87.59069342047562 41.66118220092816)
3,-87.574054,41.661182,POINT (-87.57405434133048 41.66118220092816)
4,-87.557415,41.661182,POINT (-87.55741526218534 41.66118220092816)


### Business Description fix
'Preparation of Food and Dining on Premise With Seating' => food_prep_onsite_seating  / food_prep_onsite_noseating
'Sale of Food Prepared Onsite With Dining Area' / 'Sale of Food Prepared Onsite Without Dining Area' => onsite_food_prep_wDining
'Retail Sales of Perishable Foods' => perishable_food / nonperishable_food
'Consumption of Liquor on Premises' => liquor_served


In [5]:
# reverse geocode - provide lat/long
# Get: name, suburb, result_type, postcode, rank[popularity], place_id   
# Not needed use details

# Details provide place_id
# Get: categories, amenity, cuisine, addr:

# places provide address and radius and category or name (could search for dunkin donuts, starbucks**)
# Get nearby restaurants (type), wheel chair accessible, dogs allowed,

In [47]:
#Geoapify - 3000 queries per day
# AUTHENTICATION = f'&format=json&apiKey={geoapify_key}'

all_data = []
for i in grid_layout.iterrows():

    # query parameters
    long,lat = i[1].lat,i[1].lon
    categories = "catering.restaurant"
    radius = 1200
    filters = f"circle:{long},{lat},{radius}"
    bias = f"proximity:{long},{lat}"
    limit = 200

    REQUEST_PARAMS = {
        "categories":categories,
        "limit":limit,
        "filter":filters,
        "bias":bias,
        "apiKey":geoapify_key    
    }

    # get business info - should I use places-details ??
    places_url = f'https://api.geoapify.com/v2/places'

    geo_data = requests.get(places_url, params=REQUEST_PARAMS).json()

    # Print the json (pretty printed)
    # print(json.dumps(geo_data, indent=4, sort_keys=True))
    all_data.append(geo_data)


Audio(sound_file, autoplay=True)

InvalidIndexError: (slice(0, 1, None), slice(None, None, None))

In [49]:
all_data[4]['features'][0]['properties']

{'name': 'Chicago Pita Kitchen',
 'country': 'United States',
 'country_code': 'us',
 'state': 'Illinois',
 'county': 'Cook County',
 'city': 'Chicago',
 'municipality': 'Hyde Park Township',
 'postcode': '60633',
 'suburb': 'Hegewisch',
 'street': 'South Brainard Avenue',
 'housenumber': '13227',
 'lon': -87.55396332945435,
 'lat': 41.6550718,
 'state_code': 'IL',
 'formatted': 'Chicago Pita Kitchen, 13227 South Brainard Avenue, Chicago, IL 60633, United States of America',
 'address_line1': 'Chicago Pita Kitchen',
 'address_line2': '13227 South Brainard Avenue, Chicago, IL 60633, United States of America',
 'categories': ['building',
  'building.catering',
  'catering',
  'catering.restaurant',
  'catering.restaurant.greek'],
 'details': ['details',
  'details.building',
  'details.catering',
  'details.contact'],
 'datasource': {'sourcename': 'openstreetmap',
  'attribution': '© OpenStreetMap contributors',
  'license': 'Open Database Licence',
  'url': 'https://www.openstreetmap.or

In [52]:
restaurants_only = []
for region in all_data:
    for i in region['features']:
        restaurants_only.append(i['properties'])

In [55]:
# Chicago is suppose to have around 7,300 restaurants
len(restaurants_only)

{'name': 'Taco Mex',
 'country': 'United States',
 'country_code': 'us',
 'state': 'Illinois',
 'county': 'Cook County',
 'city': 'Chicago',
 'municipality': 'Hyde Park Township',
 'postcode': '60617',
 'suburb': 'South Deering',
 'street': 'South Torrence Avenue',
 'housenumber': '10658',
 'lon': -87.55996449615577,
 'lat': 41.701098599999995,
 'state_code': 'IL',
 'formatted': 'Taco Mex, 10658 South Torrence Avenue, Chicago, IL 60617, United States of America',
 'address_line1': 'Taco Mex',
 'address_line2': '10658 South Torrence Avenue, Chicago, IL 60617, United States of America',
 'categories': ['building',
  'building.catering',
  'catering',
  'catering.restaurant',
  'catering.restaurant.mexican'],
 'details': ['details',
  'details.building',
  'details.catering',
  'details.contact',
  'details.facilities'],
 'datasource': {'sourcename': 'openstreetmap',
  'attribution': '© OpenStreetMap contributors',
  'license': 'Open Database Licence',
  'url': 'https://www.openstreetmap.